# Pulling hometown demographic data

Potentially useful census variables from summary file 1:

P013001: median age

P017001: average household size

P001001: Total population

P012010: Total!!Male!!22 to 24 years

P012011: Total!!Male!!25 to 29 years

P012012: Total!!Male!!30 to 34 years

P012034: 	Total!!Female!!22 to 24 years

P012035: Total!!Female!!25 to 29 years

P012036: 	Total!!Female!!30 to 34 years

P013002: Median age!!Male	

P013003: Median age!!Female	

P017001: Average household size!!Total	

P037001: Average family size!!Total	

ACS5 Variable Definitions: https://api.census.gov/data/2016/acs/acs5/variables.html

Potentially useful census variables from ACS5:

B01001: Sex by age

B06009: PLACE OF BIRTH BY EDUCATIONAL ATTAINMENT IN THE UNITED STATES

B05010: RATIO OF INCOME TO POVERTY LEVEL IN THE PAST 12 MONTHS BY NATIVITY OF CHILDREN UNDER 18 YEARS IN FAMILIES AND SUBFAMILIES BY LIVING ARRANGEMENTS AND NATIVITY OF PARENTS

B05010_001E: Estimate!!Total

B05010_002E: Estimate!!Total!!Under 1.00

B05010_010E: Estimate!!Total!!1.00 to 1.99

B05010_018E: Estimate!!Total!!2.0 and over

B07011: MEDIAN INCOME IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS) BY GEOGRAPHICAL MOBILITY IN THE PAST YEAR FOR CURRENT RESIDENCE IN THE UNITED STATES

B07011_001E: 	Estimate!!Median income in the past 12 months!!Total

B08006: 	SEX OF WORKERS BY MEANS OF TRANSPORTATION TO WORK

B08303: TRAVEL TIME TO WORK



Demographic variables to get for hometowns:
  - Median income
  - Educational attainment
  - Ratio of young men to young women (22-34)

Other possibilities:
  - Travel time to work
  - Average household size
  - Distance between hometowns
  - Population density

In [42]:
# get some ACS5 data
import requests
HOST = 'https://api.census.gov/data'
year='2018'
dataset='acs/acs5'
base_url = '/'.join([HOST, year, dataset])
get_vars = ['NAME', 'B05010_001E']
predicates = {}
predicates['get'] = ','.join(get_vars)
predicates['for'] = '06075'
r = requests.get(base_url, params=predicates)
print(r.text)

error: invalid 'for' argument


In [43]:
import geopy
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent='djh-test')

def get_lat_long(town):
    location = geolocator.geocode(town)
    latitude = location.latitude
    longitude = location.longitude
    return latitude, longitude

def get_fips(latitude, longitude):
    base_url = 'https://geo.fcc.gov/api/census/block/find?'
    fcc_api_url = '&'.join([base_url, f'latitude={latitude}', f'longitude={longitude}', 'format=json'])
    r = requests.get(fcc_api_url)
    county_fips = r.json()['County']['FIPS']
    return county_fips

In [44]:
# Test for a few 
#print(get_fips('Miami, Florida'))
#print(get_fips('Wagener, South Carolina'))
#print(get_fips('Loveland, Colorado'))
#print(get_fips('Richmond, VA'))
#print(get_fips('London, England'))

In [45]:
def get_census_data_for_fips(fips):
    HOST = 'https://api.census.gov/data'
    year='2018'
    dataset='acs/acs5'
    base_url = '/'.join([HOST, year, dataset])
    get_vars = ['NAME', 'B05010_001E']
    predicates = {}
    predicates['get'] = ','.join(get_vars)
    predicates['for'] = f'for=county:{fips[2:]}&in=state:{fips[:2]}'
    url = base_url+'?get='+predicates['get']+'&'+predicates['for']
    #url = '&'.join([base_url, predicates['get'], predicates['for']])
    r = requests.get(url)
    return r.text

def get_census_data_for_town(town):
    latitude, longitude = get_lat_long(town)
    fips = get_fips(latitude, longitude)
    data = get_census_data_for_fips(fips)
    return data

In [52]:
data = get_census_data_for_town('San Francisco, California')

In [53]:
print(data)

[["NAME","B05010_001E","state","county"],
["San Francisco County, California","111951","06","075"]]
